In [1]:
# Importar librerías
import pandas as pd
import os

# Establecer directorio de trabajo
os.chdir(r'/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets_Codigo')

# Leer archivos CSV procesados
df_clifd = pd.read_csv('Mer_Clifd.txt', encoding="latin1")
df_clird = pd.read_csv('Mer_Clird.txt', encoding='latin1')
df_clied = pd.read_csv('Mer_clied.txt', encoding='latin1')  
df_clifad = pd.read_csv('Mer_Clifad.txt', encoding='latin1')
df_clind = pd.read_csv('Mer_Clind.txt', encoding='latin1')

# Cambiar directorio de trabajo
os.chdir(r'/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets')

# Leer archivos CSV originales
df_articulos = pd.read_csv('Articulos.txt', encoding='latin1')
df_profd = pd.read_csv('Profd.txt', encoding="latin1")
df_prof = pd.read_csv('Prof.txt', encoding="latin1")

In [2]:
# Concatenar DataSets procesados
df_merged = pd.concat([df_clifd, df_clird, df_clied, df_clifad]) 
# Llenar fecha y eliminar columnas redundantes
df_merged['Fec'] = df_merged['Fecfac'].fillna(df_merged['Fecrem']).fillna(df_merged['Fecncre'])
df_merged.drop(['Fecfac', 'Fecrem', 'Fecncre'], axis=1, inplace=True)
df_merged

,Claart,Cant,Clacli,Clatipcli,Fec
0,01CEMCAZ50,40.0,PSP14,0.0,01/09/20
1,01CEMCAZ50,100.0,RELP55,1.0,31/08/20
2,01CEMCAZ50,100.0,RELP55,1.0,31/08/20
3,01SON40,18.0,RELP55,1.0,31/08/20
4,GTRAS,1.0,RELP55,1.0,31/08/20
...,...,...,...,...,...
6042,05COPL34,5.0,MRSZI0809,0.0,10/11/23
6043,05CODPL34,4.0,MRSZI0809,0.0,11/11/23
6044,05TUBPL34,2.0,MRSZI0809,0.0,11/11/23
6045,01CEMBIG,2.0,MRS409,9.0,13/11/23


In [3]:
# Convertir fecha a datetime
df_merged['Fec'] = pd.to_datetime(df_merged['Fec'], format="%d/%m/%y")

# Filtrar por fecha de referencia 
fecha_reciente = df_merged['Fec'].max()
fecha_referencia = fecha_reciente - pd.Timedelta(days=30)
df_filtrado = df_merged[df_merged['Fec'] >= fecha_referencia].copy()

# Formatear fecha
df_filtrado['Fec'] = df_filtrado['Fec'].dt.strftime('%d/%m/%y')

In [4]:
# Agrupar y sumar cantidad
df_cant = df_filtrado.groupby('Claart')['Cant'].sum().reset_index()

# Contar días únicos
df_dias = df_filtrado.groupby('Claart')['Fec'].nunique().reset_index()
df_dias = df_dias.rename(columns={'Fec': 'Dias_Totales'})

# Unir dataframes
df_merall = df_cant.merge(df_dias, on='Claart')

In [5]:
# Obtener proveedor de los articulos
df_merged = df_profd.groupby('Claart')['Numfac'].max().reset_index()
df_merged = pd.merge(df_merged, df_profd, on=['Claart', 'Numfac'], how='left')
df_result = pd.merge(df_merged, df_prof, on='Numfac', how='left')[['Claart', 'Clapro']]
df_final = pd.merge(df_merall, df_result, on='Claart', how='left')

In [6]:
# Filtrar por especial = 'no'
df_final = pd.merge(df_final, df_articulos[['Claart', 'especial']], how='left', on='Claart')
df_merall = df_final.loc[df_final['especial'] == 'no'][['Claart', 'Cant', 'Dias_Totales', 'Clapro']]

In [7]:
# Convertir fecha a datetime
df_clind['Fecncre'] = pd.to_datetime(df_clind['Fecncre'], format="%d/%m/%y")

# Filtrar por fecha de referencia (DataSet Clind)
df_clind_filtrado = df_clind[df_clind['Fecncre'] >= fecha_referencia].copy()

# Formatear fecha
df_clind_filtrado['Fecncre'] = df_clind_filtrado['Fecncre'].dt.strftime('%d/%m/%y')

# Agrupar y sumar cantidad (DataSet Clind)
df_cant = df_clind_filtrado.groupby('Claart')['Cant'].sum().reset_index()

# Filtrar por especial = 'no' (DataSet Clind)
df_final = pd.merge(df_cant, df_articulos[['Claart', 'especial']], how='left', on='Claart')
df_clind = df_final.loc[df_final['especial'] == 'no'][['Claart', 'Cant']]

In [8]:
# Restar cantidad de Clind a la concatenación de Clifd, Clird, clied y Clifad y Clind
df_merall = pd.merge(df_merall, df_clind[['Claart', 'Cant']], on='Claart', how='left')
df_merall['Cant_y'] = df_merall['Cant_y'].fillna(0)
df_merall['Cant'] = df_merall['Cant_x'] - df_merall['Cant_y']
df_merall = df_merall.drop(['Cant_x', 'Cant_y'], axis=1)
df_merall = df_merall[['Claart', 'Cant', 'Dias_Totales', 'Clapro']]

In [ ]:
# Guardar dataframe
df_merall.to_csv(r'/Users/adrianamedrano/Documents/Ariana/Residencias/PrototipoResidencias/DataSets_Codigo/MerAll.txt', index=False, encoding="latin1")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6e53d21-17bd-40b4-9b5d-0d63804f9374' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>